# Ultimate TTT Model

## Setup

In [1]:
import tensorflow as tf
import keras
from keras import layers
import scipy.signal

# used for fps logging
from datetime import datetime

# sanity check
tf.config.list_physical_devices()

2023-06-23 12:42:26.217276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 12:42:26.805291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-23 12:42:27.443819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 12:42:27.460376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Env

In [2]:
# used to send data
import os
import time
import socket

# proto definitions
import py.board_pb2 as pb

# misc
from typing import Tuple

# math
import numpy as np

# board constants
ROWS = 3
COLS = 3
CELLS = 9

# socket constants
S_PORT = 8000
A_PORT = 8001
R_PORT = 8002
MAX_MSG_SIZE = 512

# reward parameters
WIN_REWARD = 10
CELL_REWARD = 2
INVALID_PENALTY = -10

# debugging
used_actions = dict()
cur_state = None


class UltimateTicTacToeEnv:
    obs_dim = (9, 9, 4)
    n_actions = CELLS * CELLS

    def __init__(self) -> None:
        self.s_conn, self.a_conn, self.r_conn = None, None, None
        self.reset()

    def _receive(self, conn: socket.socket, tp: type):
        ret = tp()
        b = conn.recv(MAX_MSG_SIZE)
        ret.ParseFromString(b)
        return ret

    def _get_return(self) -> pb.ReturnMessage:
        return self._receive(self.r_conn, pb.ReturnMessage)

    def _get_state(self) -> pb.StateMessage:
        return self._receive(self.s_conn, pb.StateMessage)

    def _make_coord(self, idx) -> pb.Coord:
        return pb.Coord(row=idx // COLS, col=idx % COLS)

    def _send_action(self, move) -> None:
        action = pb.ActionMessage(move=move)
        self.a_conn.send(action.SerializeToString())

    def _to_idx(self, coord: pb.Coord) -> int:
        return coord.row * COLS + coord.col

    def _to_multi_idx(self, move: pb.Move) -> int:
        return self._to_idx(move.large) * CELLS + self._to_idx(move.small)

    def _process_state(self, state: pb.StateMessage) -> np.ndarray:
        """
        The structure of the state:
        (9, 9, 4)
        Outer 9 represent board cells
        inner 9 represent the cell spaces
        each space has 3 objects:
            space owner (0, 1, 2) representing if the space is claimed or not
            cell owner (0, 1, 2) representing if the cell the space belongs to is claimed or not
            curcellornot (0, 1); 1 if the space belongs to the current cell, 0 if not
            turn (1, 2) 1 if the current turn is player1, 2 if the current turn is player2
        """
        board_state = np.zeros(self.obs_dim)
        for cell_idx in range(len(state.board.cells)):
            for space_idx in range(len(state.board.cells[cell_idx].spaces)):
                board_state[cell_idx, space_idx, 0] = (
                    state.board.cells[cell_idx].spaces[space_idx].val
                )
                board_state[cell_idx, space_idx, 1] = state.cellowners[cell_idx]
                board_state[cell_idx, space_idx, 2] = (
                    1 if self._to_idx(state.board.curCell) == cell_idx else 0
                )
                board_state[cell_idx, space_idx, 3] = state.turn

        return board_state

    def _get_exploration_reward(self, action: int, msg: pb.ReturnMessage) -> float:
        if msg.valid:
            return 0
        return INVALID_PENALTY

    def _get_win_reward(self, msg: pb.ReturnMessage) -> float:
        """
        Get's the reward for winning if the game was won
        """
        # the turn sent in the return message should still be the caller's turn
        if msg.state.winner == msg.state.turn:
            return WIN_REWARD
        return 0

    def _get_cell_reward(self, msg: pb.ReturnMessage) -> float:
        """
        Get's the reward for claiming a cell if a cell was claimed
        """
        if self.prev_cellowners == msg.state.cellowners:
            return 0
        elif list(msg.state.cellowners).count(
            msg.state.turn
        ) > self.prev_cellowners.count(msg.state.turn):
            self.prev_cellowners = list(msg.state.cellowners)
            return CELL_REWARD
        return 0

    def _get_reward(self, action: pb.Move, msg: pb.ReturnMessage) -> float:
        return (
            self._get_exploration_reward(action, msg)
            + self._get_cell_reward(msg)
            + self._get_win_reward(msg)
        )
    
    def _reset_vars(self):
        self.prev_cellowners = [pb.NONE] * 9
        self.stop_when_invalid = True

        self.s_conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.a_conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.r_conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.s_conn.connect(("", 8000))
        self.a_conn.connect(("", 8001))
        self.r_conn.connect(("", 8002))

    # public section
    def observe(self) -> np.ndarray:
        global cur_state
        state = self._get_state()
        cur_state = state
        self._turn = state.turn
        return self._process_state(state)

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool]:
        """
        Returns:
            - next state
            - reward for the action
            - done / not done
            - valid / invalid
        """
        # update the count
        if action not in used_actions:
            used_actions[action] = 1
        else:
            used_actions[action] += 1

        # send action and get response
        self._send_action(self.to_move(action))
        ret_message = self._get_return()

        # return information
        reward = self._get_reward(action, ret_message)
        done = ret_message.state.done
        if not done:
            if self.stop_when_invalid:
                # stop the episode if the move was invalid
                return self.observe(), reward, not ret_message.valid, ret_message.valid
            else:
                return self.observe(), reward, done, ret_message.valid
        else:
            return (
                self._process_state(ret_message.state),
                reward,
                done,
                ret_message.valid,
            )

    def turn(self):
        return self._turn

    def reset(self) -> np.ndarray:
        while 1:
            try:
                self.cleanup()
                self.pid = os.spawnl(os.P_NOWAIT, "uttt", "uttt", "aivai")
                time.sleep(0.12)
                self._reset_vars()
                break
            except ConnectionRefusedError:
                pass
        return self.observe()

    def cleanup(self):
        os.system("killall -q uttt")
        if self.s_conn is not None:
            self.s_conn.close()
            self.r_conn.close()
            self.a_conn.close()

    def __del__(self):
        self.cleanup()

    def to_move(self, idx: int) -> pb.Move:
        outer_idx = idx // CELLS
        inner_idx = idx % CELLS

        return pb.Move(
            large=self._make_coord(outer_idx), small=self._make_coord(inner_idx)
        )

In [3]:
env = UltimateTicTacToeEnv()

## Buffers and Losses

In [4]:
# Hyperparameters of the PPO algorithm
clip_ratio = 0.2
target_kl = 0.01

In [5]:
def discounted_cumulative_sums(x, discount):
    # Discounted cumulative sums of vectors for computing rewards-to-go and advantage estimates
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]


class Buffer:
    # Buffer for storing trajectories
    def __init__(self, observation_dimensions, size, gamma=0.99, lam=0.95):
        # Buffer initialization
        self.observation_buffer = np.zeros(
            (size, *observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros(size, dtype=np.int32)
        self.advantage_buffer = np.zeros(size, dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.return_buffer = np.zeros(size, dtype=np.float32)
        self.value_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.pointer, self.trajectory_start_index = 0, 0

    def store(self, observation, action, reward, value, logprobability):
        # Append one step of agent-environment interaction
        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.value_buffer[self.pointer] = value
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1

    def finish_trajectory(self, last_value=0):
        # Finish the trajectory by computing advantage estimates and rewards-to-go
        path_slice = slice(self.trajectory_start_index, self.pointer)
        rewards = np.append(self.reward_buffer[path_slice], last_value)
        values = np.append(self.value_buffer[path_slice], last_value)

        deltas = rewards[:-1] + self.gamma * values[1:] - values[:-1]

        self.advantage_buffer[path_slice] = discounted_cumulative_sums(
            deltas, self.gamma * self.lam
        )
        self.return_buffer[path_slice] = discounted_cumulative_sums(
            rewards, self.gamma
        )[:-1]

        self.trajectory_start_index = self.pointer

    def get(self):
        # Get all data of the buffer and normalize the advantages
        self.pointer, self.trajectory_start_index = 0, 0
        advantage_mean, advantage_std = (
            np.mean(self.advantage_buffer),
            np.std(self.advantage_buffer),
        )
        self.advantage_buffer = (self.advantage_buffer - advantage_mean) / advantage_std
        return (
            self.observation_buffer,
            self.action_buffer,
            self.advantage_buffer,
            self.return_buffer,
            self.logprobability_buffer,
        )


def logprobabilities(logits, a):
    # Compute the log-probabilities of taking actions a by using the logits (i.e. the output of the actor)
    logprobabilities_all = tf.nn.log_softmax(logits)
    logprobability = tf.reduce_sum(
        tf.one_hot(a, UltimateTicTacToeEnv.n_actions) * logprobabilities_all, axis=1
    )
    return logprobability


# Sample action from actor
@tf.function
def sample_action(observation, actor):
    logits = actor(observation)
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
    return logits, action


# Train the policy by maxizing the PPO-Clip objective
@tf.function
def train_policy(
    observation_buffer,
    action_buffer,
    logprobability_buffer,
    advantage_buffer,
    actor: keras.Model,
    policy_optimizer: tf.keras.optimizers.Optimizer,
):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        ratio = tf.exp(
            logprobabilities(actor(observation_buffer), action_buffer)
            - logprobability_buffer
        )
        min_advantage = tf.where(
            advantage_buffer > 0,
            (1 + clip_ratio) * advantage_buffer,
            (1 - clip_ratio) * advantage_buffer,
        )

        policy_loss = -tf.reduce_mean(
            tf.minimum(ratio * advantage_buffer, min_advantage)
        ) + tf.reduce_sum(actor.losses)
    policy_grads = tape.gradient(policy_loss, actor.trainable_variables)
    policy_optimizer.apply_gradients(zip(policy_grads, actor.trainable_variables))

    kl = tf.reduce_mean(
        logprobability_buffer
        - logprobabilities(actor(observation_buffer), action_buffer)
    )
    kl = tf.reduce_sum(kl)
    return kl


# Train the value function by regression on mean-squared error
@tf.function
def train_value_function(
    observation_buffer,
    return_buffer,
    critic: keras.Model,
    value_optimizer: tf.keras.optimizers.Optimizer,
):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation.
        value_loss = tf.reduce_mean(
            (return_buffer - critic(observation_buffer)) ** 2
        ) + tf.reduce_sum(critic.losses)
    value_grads = tape.gradient(value_loss, critic.trainable_variables)
    value_optimizer.apply_gradients(zip(value_grads, critic.trainable_variables))

## Model

In [6]:
load_model = True

In [7]:
def create_actor():
    # model inputs
    inputs = tf.keras.Input(shape=UltimateTicTacToeEnv.obs_dim)

    x = layers.Conv1D(
        256,
        kernel_size=9,
        strides=1,
        padding="valid",
        activation="relu",
        activity_regularizer="l2",
    )(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(
        1024,
        kernel_size=(9, 1),
        strides=(1, 1),
        padding="valid",
        activation="relu",
        activity_regularizer="l2",
    )(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)

    logits = tf.keras.layers.Dense(UltimateTicTacToeEnv.n_actions)(x)
    return tf.keras.Model(inputs=inputs, outputs=logits)


def create_critic():
    inputs = tf.keras.Input(shape=UltimateTicTacToeEnv.obs_dim)

    x = layers.Conv1D(
        256,
        kernel_size=9,
        strides=1,
        padding="valid",
        activation="relu",
        activity_regularizer="l2",
    )(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(
        1024,
        kernel_size=(9, 1),
        strides=(1, 1),
        padding="valid",
        activation="relu",
        activity_regularizer="l2",
    )(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)

    values = tf.keras.layers.Dense(1)(x)
    return tf.keras.Model(inputs=inputs, outputs=values)


if load_model and os.path.exists("actor1.keras") and os.path.exists("critic1.keras"):
    print("loading model...")
    actor1 = tf.keras.models.load_model("actor1.keras")
    critic1 = tf.keras.models.load_model("critic1.keras")
else:
    print("creating model...")
    actor1 = create_actor()
    critic1 = create_critic()

loading model...


2023-06-23 12:42:27.655991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 12:42:27.656169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-23 12:42:27.656273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
actor1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 4)]         0         
                                                                 
 conv1d (Conv1D)             (None, 9, 1, 256)         9472      
                                                                 
 dropout (Dropout)           (None, 9, 1, 256)         0         
                                                                 
 conv2d (Conv2D)             (None, 1, 1, 1024)        2360320   
                                                                 
 dropout_1 (Dropout)         (None, 1, 1, 1024)        0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              104960

In [9]:
critic1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9, 9, 4)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 1, 256)         9472      
                                                                 
 dropout_3 (Dropout)         (None, 9, 1, 256)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 1024)        2360320   
                                                                 
 dropout_4 (Dropout)         (None, 1, 1, 1024)        0         
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              1049

## Optimizers

In [10]:
# learning rate hyperparams
policy_learning_rate = 1e-4  # 3e-4
value_function_learning_rate = 8e-4  # 1e-3

In [11]:
actor1_optimizer = tf.keras.optimizers.Adam(learning_rate=policy_learning_rate)
critic1_optimizer = tf.keras.optimizers.Adam(learning_rate=value_function_learning_rate)

## Train

In [12]:
import tensorboard

In [13]:
# Define training hyperparameters

# alg hyperparameters
gamma = 0.99
lam = 0.97

# training time hyperparameters
train_policy_iterations = 30
train_value_iterations = 30
epochs = 30*3*5  # 25 * 4096 * 2 ~ 1 hr
minibatch_size = 128
steps_per_epoch = 2048  # should be a multiple of minibatch_size

In [14]:
epoch = 119
summary_writer = tf.summary.create_file_writer(f"./logs/ppo")

In [15]:
buffer = Buffer(UltimateTicTacToeEnv.obs_dim, steps_per_epoch, gamma=gamma, lam=lam)

In [16]:
# main training function
def train_model(
    actor, critic, actor_optimizer, critic_optimizer, player1, epoch_start
) -> int:
    if player1:
        cond = lambda turn: turn == pb.Owner.PLAYER1
    else:
        cond = lambda turn: turn == pb.Owner.PLAYER2

    # Iterate over the number of epochs
    for epoch in range(epoch_start, epoch_start + epochs):
        # Initialize the sum of the returns, lengths and number of episodes for each epoch
        sum_return = 0
        sum_length = 0
        num_episodes = 0
        episode_return = 0
        episode_length = 0

        # logging variables
        num_valid = 0
        num_wins = 0

        # Iterate over the steps of each epoch
        observation = env.reset()
        t = 0
        start_time = datetime.now()
        while t < steps_per_epoch:
            if cond(env.turn()):  # if it's the training ai's turn
                env.stop_when_invalid = True
                # Get the logits, action, and take one step in the environment
                observation = observation.reshape(1, *env.obs_dim)
                logits, action = sample_action(observation, actor)
                observation_new, reward, done, valid = env.step(action[0].numpy())
                episode_return += reward
                episode_length += 1

                # logging variables
                num_valid += 1 if valid else 0
                num_wins += 1 if reward >= WIN_REWARD else 0

                # Get the value and log-probability of the action
                value_t = critic(observation)
                logprobability_t = logprobabilities(logits, action)

                # Store obs, act, rew, v_t, logp_pi_t
                buffer.store(observation, action, reward, value_t, logprobability_t)

                t += 1
            else:
                env.stop_when_invalid = False
                # just keep trying random actions until it works
                observation_new, reward, done, valid = env.step(
                    np.random.randint(0, env.n_actions)
                )

            # Update the observation
            observation = observation_new

            # Finish trajectory if reached to a terminal state
            terminal = done
            if terminal or t == steps_per_epoch - 1:
                last_value = 0 if done else critic(observation.reshape(1, *env.obs_dim))
                buffer.finish_trajectory(last_value)
                sum_return += episode_return
                sum_length += episode_length
                num_episodes += 1
                observation, episode_return, episode_length = env.reset(), 0, 0

            print(
                f"Step {t} / {steps_per_epoch}; "
                + f"% valid = {num_valid / (t+1)}; "
                + f"fps: {t/(datetime.now()-start_time).total_seconds()}; "
                + f"win rate: {num_wins/num_episodes if num_episodes > 0 else 0}",
                end="\r",
            )
        print()

        # Get values from the buffer
        # 0 - observation_buffer,
        # 1 - action_buffer,
        # 2 - advantage_buffer,
        # 3 - return_buffer,
        # 4 - logprobability_buffer,
        (
            observation_buffer,
            action_buffer,
            advantage_buffer,
            return_buffer,
            logprobability_buffer,
        ) = buffer.get()

        # Update the policy and implement early stopping using KL divergence
        for _ in range(train_policy_iterations):
            kl = 0
            for i in range(steps_per_epoch // minibatch_size):
                kl += train_policy(
                    tf.constant(
                        observation_buffer[
                            i * minibatch_size : (i + 1) * minibatch_size
                        ]
                    ),  # obs
                    tf.constant(
                        action_buffer[i * minibatch_size : (i + 1) * minibatch_size]
                    ),  # act
                    tf.constant(
                        logprobability_buffer[
                            i * minibatch_size : (i + 1) * minibatch_size
                        ]
                    ),  # logprobs
                    tf.constant(
                        advantage_buffer[i * minibatch_size : (i + 1) * minibatch_size]
                    ),  # advantages
                    actor,
                    actor_optimizer,
                )
            kl /= steps_per_epoch // minibatch_size
            if kl > 1.5 * target_kl:
                # Early Stopping
                break

        # Update the value function
        for _ in range(train_value_iterations):
            for i in range(steps_per_epoch // minibatch_size):
                train_value_function(
                    tf.constant(
                        observation_buffer[
                            i * minibatch_size : (i + 1) * minibatch_size
                        ]
                    ),  # obs buffer
                    tf.constant(
                        return_buffer[i * minibatch_size : (i + 1) * minibatch_size]
                    ),  # returns
                    critic,
                    critic_optimizer,
                )

        # Print mean return and length for each epoch
        print(
            f"Epoch: {epoch + 1}. Mean Return: {sum_return / num_episodes}. Mean Length: {sum_length / num_episodes}"
        )
        print("=" * 64)

        # log scalars
        with summary_writer.as_default():
            tf.summary.scalar("episode/win rate", num_wins / num_episodes, step=epoch)
            tf.summary.scalar("episode/valid percentage", num_valid / t, step=epoch)
            tf.summary.scalar("episode/mean reward", sum_return / num_episodes, step=epoch)
            tf.summary.scalar("episode/mean length", sum_length / num_episodes, step=epoch)

    return epoch

In [17]:
epoch = train_model(actor1, critic1, actor1_optimizer, critic1_optimizer, True, epoch)

2023-06-23 12:42:29.336930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Step 2048 / 2048; % valid = 0.9804782820888238; fps: 50.641458891412476; win rate: 0.36842105263157896


2023-06-23 12:43:09.856455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-23 12:43:10.410055: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7faa264dd410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-23 12:43:10.410074: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-06-23 12:43:10.412897: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-23 12:43:10.503080: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch: 120. Mean Return: 6.684210526315789. Mean Length: 17.95614035087719
Step 2048 / 2048; % valid = 0.9790141532454856; fps: 49.514736243123544; win rate: 0.44915254237288146
Epoch: 121. Mean Return: 7.11864406779661. Mean Length: 17.347457627118644
Step 2048 / 2048; % valid = 0.9775500244021474; fps: 50.71705948036311; win rate: 0.398230088495575234
Epoch: 122. Mean Return: 6.389380530973451. Mean Length: 18.115044247787612
Step 2048 / 2048; % valid = 0.9799902391410444; fps: 49.60971710955088; win rate: 0.458333333333333366
Epoch: 123. Mean Return: 7.466666666666667. Mean Length: 17.058333333333334
Step 2048 / 2048; % valid = 0.9790141532454856; fps: 50.10041120499611; win rate: 0.385964912280701735
Epoch: 124. Mean Return: 6.456140350877193. Mean Length: 17.95614035087719
Step 2048 / 2048; % valid = 0.9775500244021474; fps: 47.92257957511579; win rate: 0.362903225806451666
Epoch: 125. Mean Return: 5.52. Mean Length: 16.376
Step 2048 / 2048; % valid = 0.9677891654465594; fps: 48.0

## Save

In [1]:
# remove previous
os.system("rm actor1.keras critic1.keras")
time.sleep(1)

# save
actor1.save("actor1.keras")
critic1.save("critic1.keras")

NameError: name 'os' is not defined